[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/linprog/allocation_ampl.ipynb)

In [ ]:
%pip install amplpy ampl-module-base ampl-module-highs --extra-index-url https://pypi.ampl.com

In [ ]:
import numpy as np
import pandas as pd
from amplpy import ampl_notebook
from google.colab import userdata

ampl = ampl_notebook(license_uuid=userdata.get("AMPL_CE_LICENSE"))

In [ ]:
%%ampl_eval
set stocks;
set accounts = {1..2};
set sectors;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set stocks_in_sector {sectors} within stocks;

param base_rate {accounts};
param market_value {stocks};
param sign {stock in stocks} = if market_value[stock] >= 0 then 1 else -1;
param volume {stocks};

var x {stock in stocks, accounts}
          >= min(market_value[stock], 0),<= max(market_value[stock], 0);
var gmv >= 0;
var sector_mv {sectors};
var sector_nmv {sectors} >= 0;

subject to complete {stock in stocks}:
          sum {account in accounts} x[stock, account] = market_value[stock];

subject to gmv_helper:
          sum {s in stocks} sign[s] * x[s, 1] = gmv;

subject to sector_mv_def {sector in sectors}:
          sum {stock in stocks_in_sector[sector]} x[stock, 1] = sector_mv[sector];

subject to sector_nmv_pos {sector in sectors}:
          sector_mv[sector] <= sector_nmv[sector];

subject to sector_nmv_neg {sector in sectors}:
          -sector_mv[sector] <= sector_nmv[sector];

minimize total_cost:
          sum {stock in stocks, account in accounts}
          base_rate[account] * sign[stock] * x[stock, account]
          + sum {stock in stocks}
          0.2 * max(sign[stock] * x[stock, 1] - volume[stock], 0)
          + sum {stock in stocks}
          0.1 * max(sign[stock] * x[stock, 1] - 0.05 * gmv, 0)
          + sum {sector in sectors}
          0.2 * max(sector_nmv[sector] - 0.2 * gmv, 0);

In [ ]:
π = pd.read_csv(
    "https://raw.githubusercontent.com/jburgy/jupyter"
    "/refs/heads/main/content/data/portfolio.csv",
    index_col=0,
)
# Fill-in blank ticker (ESC GCI LIBERTY INC SR COMMON STOCK)
π.rename(index={np.nan: "DUMMY"}, inplace=True)
π["market_value"] = π["Shares"] * π["Price"]
π["volume"] = π["Volume"] * π["Price"]

ampl.set_data(π[["market_value", "volume"]], "stocks")
ampl.param["base_rate"] = {1: 0.05, 2: 0.06}

stocks_in_sector = π.groupby("Sector").groups
ampl.set["sectors"] = stocks_in_sector.keys()
ampl.set["stocks_in_sector"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

In [ ]:
x = ampl.get_variable("x").get_values().to_pandas()
π.assign(
    account_1=x.loc[(π.index, 1), "x.val"].to_numpy(),
    account_2=x.loc[(π.index, 2), "x.val"].to_numpy(),
)